In [1]:
from dff.script import GLOBAL, TRANSITIONS, RESPONSE, Context, Actor, Message, MISC,PRE_RESPONSE_PROCESSING,LOCAL,PRE_TRANSITIONS_PROCESSING
import dff.script.conditions.std_conditions as cnd
import dff.script.labels as lbl
import re
import pandas as pd
import json
import tensorflow as tf
import pymorphy2
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertTokenizer, BertForSequenceClassification
from dff.context_storages import context_storage_factory
from dff.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import dff.script.responses as rsp
import psycopg2
from sqlalchemy import create_engine,MetaData,update,delete


from aiogram import Bot, types
from aiogram.dispatcher import Dispatcher
from aiogram.utils import executor

### Функции распознавания интена

In [2]:
def preprocessing(col,stop_words):
    col = re.sub('[^А-Яа-я]+',' ', col)
    col = col.lower()
    col = col.split(' ')
    temp=[]
    for word in col:
        temp2 = morph.normal_forms(word)[0]
        if temp2 not in stop_words: temp.append(temp2)
    return ' '.join(temp).strip()

def intent_check(ctx: Context, actor: Actor, *args, **kwargs):
    proba = Proba_Check(ctx.last_request.text)
    if proba > 0.55:
        intent = Intent_Classifier_Check(ctx.last_request.text)
        intent+="_flow"
        return (intent, "start")
    return ("global_flow","start_node")

def Intent_Classifier_Check(inp):
    temp=intent_model.predict(vectorizer.transform([preprocessing(inp,cleared_stop_words_intent)]))
    return temp[0]

def Proba_Check(inp):
    temp=intent_model.predict_proba(vectorizer.transform([preprocessing(inp,cleared_stop_words_intent)]))
    return max(temp[0])

def ir_data_collection(ctx: Context, actor: Actor, *args, **kwargs):
    request = ctx.last_request.text
    cleared = preprocessing(request,cleared_stop_words_intent)
    if cleared not in df['Cleared'].tolist() and cleared != '':
        try:
            ctx.misc['Intent_data']+=[request]
        except:
            ctx.misc['Intent_data']=[request]
    return ctx


def mapping(idx,map_data):
    if df['Класс'][idx] is None:
        update = ''
        print('Cтрока: ',df['Данные'][idx],'\nВведите цифру типа данных: 1.Соискатель, 2.Работодатель, 3.Вопрос, 4.Ни один из перечисленных.')
        data_type_num = input()
        while int(data_type_num) not in [1,2,3,4]:
            print('Такого класса нет, выберите цифру класса из сообщения выше.')
            data_type_num = input()
        if int(data_type_num) == 1:
            data_type = 'applicant'
        if int(data_type_num) == 2:
            data_type = 'employer'
        if int(data_type_num) == 3:
            data_type = 'qna'
        if int(data_type_num) == 4:
            data_type = 'remove'
        map_data[df['ID_Распознавание_Намерений'][idx]] = data_type

### Функции распознавания оскорблений

In [3]:
def toxicity_check(ctx: Context, actor: Actor, *args, **kwargs):
    if ctx.last_request.text == None: return False
    raw_class = tox_model(tokenizer.encode(ctx.last_request.text, return_tensors='pt'))
    tensor = raw_class.logits
    tensor_as_list = tensor.tolist()[0]
    return tensor_as_list[0] < tensor_as_list[1]

def toxic_check(ctx: Context, actor: Actor, *args, **kwargs):
    if 'toxic' not in ctx.misc:
        ctx.misc['toxic'] = ctx.last_request.text
    else:
        ctx.misc['toxic'] += ' | ' + ctx.last_request.text
    return ctx

### Функции ответов на вопросы

In [4]:
def qna_last_resort(clear_req):
    if clear_req.find('ваканс') != -1:
        return 'Вы можете посмотреть вакансии, на нашем сайте в разделе вакансии.'
    time_n_place = ['работ','открыв','находи','попаст','проехат','график','расписан']
    for word in time_n_place:
        if clear_req.find(word) != -1:
            return 'Мы находися по адресу г.Ярославль Урочская улица, 44. Работаем с 10 до 17 часов в будние дни.'
    contacts = ['телеф', 'номер','почт', 'связ', 'контакт']
    for word in  contacts:
        if clear_req.find(word) != -1:
            return 'Наш телефон: +7(968)958-2664.Наша электронная почта: mironov.yar@gmail.com'
    return ''

def qna_func(ctx: Context, actor: Actor, *args, **kwargs):
    req = re.sub('[^А-Яа-я]+',' ', ctx.last_request.text)
    req = req.lower().split(' ')
    temp=[]
    for word in req:
        if morph.normal_forms(word)[0] not in stop_qna_words_clear:
            temp.append(morph.normal_forms(word)[0])
    clear_req =' '.join(temp).strip()
    if clear_req in qna['Cleared'].tolist():
        idx = qna['Ответ'][qna['Cleared']==clear_req].index[0]
        return Message(text = str(qna['Ответ'][idx]))
    if qna_last_resort(clear_req) == '':
        return Message(text = 'К сожалению, я не понимаю ваше вопроса, пожалуйста перефразируйте его.')
    else:
        return Message(text = str(qna_last_resort(clear_req)))

### Анкетирование клиентов
## Общие

In [5]:

def misc_key_catcher(ctx: Context, actor: Actor, *args, **kwargs):
    ctx.misc['temp'] = ctx.last_label[1]
    return ctx

def misc_item_catcher(ctx: Context, actor: Actor, *args, **kwargs):
    key = ctx.misc['temp']
    processed_item = ctx.last_request
    ctx.misc[key] = processed_item.text
    return ctx

def client_type(ctx: Context, actor: Actor, *args, **kwargs):
    if bool(re.search(ctx.last_request.text,r'\bДа\b',re.IGNORECASE)):
        if ctx.last_label[0] == "applicant_flow":
            ctx.misc['type'] = 'applicant'
        elif ctx.last_label[0] ==  "employer_flow":
            ctx.misc['type'] = 'employer'
    if 'temp' in ctx.misc:
        ctx.misc.pop('temp')  
    return ctx
                         
def remove_misc(ctx: Context, actor: Actor, *args, **kwargs):
    if bool(ctx.misc):
        if 'toxic' in ctx.misc:
            temp = ctx.misc['toxic']
            ctx.misc.clear()
            ctx.misc['toxic'] = temp
        else:
            ctx.misc.clear()
    return ctx

def num_check(ctx: Context, actor: Actor, *args, **kwargs):
    if bool(ctx.last_label):
        try:
            last_request = int(ctx.last_request.text)
        except:
            last_request = ctx.last_request.text
        if ctx.last_label[0] == "applicant_flow":
             return last_request in applicant_dict
        elif ctx.last_label[0] == "employer_flow":
            return last_request in employer_dict
    return False

def type_check(ctx: Context, actor: Actor, *args, **kwargs):
    if 'type' in ctx.misc:
        return True
    return False

#### Только для соискателей
Стоит объеденить функции

In [6]:
def print_applicant(ctx: Context, actor: Actor, *args, **kwargs):
    if bool(ctx.misc):
        return Message(text = f"""Пожалуйста проверьте данные
        1.Имя: {ctx.misc['name']}
        2.Пол: {ctx.misc['gender']}
        3.Возраст: {ctx.misc['birthdate']} 
        4.Желаемая должность: {ctx.misc['profession']}
        5.Стаж работы: {ctx.misc['experience']}
        6.Основное образование: {ctx.misc['education']}
        7.Желаемая заработная плата:{ctx.misc['salary']}
        8.Желаемый тип занятости: {ctx.misc['employment_type']}
        9.Контактные данные:{ctx.misc['contacts']}
        Все ли верно?(Да/Нет)""")
    return Message(text = '')

def redo_response(ctx: Context, actor: Actor, *args, **kwargs): 
    try:
        last_request = int(ctx.last_request.text)
    except:
        last_request = ctx.last_request.text 
    if last_request in applicant_dict: 
        return Message(text=applicant_dict[last_request][0])
    else:
        return Message()

def redo_key_catcher(ctx: Context, actor: Actor, *args, **kwargs):
    try:
        last_request = int(ctx.last_request.text)
    except:
        last_request = ctx.last_request.text 
    if last_request in applicant_dict: 
        ctx.misc['temp'] = applicant_dict[last_request][1]
    return ctx


### Только для работодаетелй

In [7]:
def print_employer(ctx: Context, actor: Actor, *args, **kwargs):
    if bool(ctx.misc):
        return Message(text = f"""Пожалуйста проверьте данные
        1.Ваше ФИО: {ctx.misc['ename']}
        2.Название организции: {ctx.misc['orgname']} 
        3.Контактные данные: {ctx.misc['econtacts']}
        4.Должность: {ctx.misc['eprofession']}
        5.Примерная заработная плата: {ctx.misc['esalary']}
        6.Требования к сотруднику:\n{ctx.misc['demands']}
        Все ли верно?(Да/Нет)""")
    return Message(text = 'Произошла ошибка')

def redo_response_emp(ctx: Context, actor: Actor, *args, **kwargs):
    try:
        last_request = int(ctx.last_request.text)
    except:
        last_request = ctx.last_request.text 
    if last_request in employer_dict: 
        return Message(text=employer_dict[last_request][0])
    else:
        return Message()

def redo_key_catcher_emp(ctx: Context, actor: Actor, *args, **kwargs):
    try:
        last_request = int(ctx.last_request.text)
    except:
        last_request = ctx.last_request.text 
    if last_request in employer_dict: 
        ctx.misc['temp'] = employer_dict[last_request][1]
    return ctx

In [8]:
morph = pymorphy2.MorphAnalyzer()
#Словарь стоп слов для IR
stop_words_intent = ['Добрый','день','вечер','ночь','хотим','Здраствуйте','Привет','c','лол','и','не','эх','хотеться','бы','надо','вы','найти','можно','у','а','да','нет','хай']
cleared_stop_words_intent = []
for word in stop_words_intent:
    cleared_stop_words_intent.append(morph.normal_forms(word)[0])
    
alchemyEngine = create_engine('xxx', pool_recycle=3600); #удален путь к БД
meta = MetaData(bind=alchemyEngine)
MetaData.reflect(meta)
intent_data = meta.tables["Распознавание_Намерений"]

dbConnection = alchemyEngine.connect();




map_data={}
df = pd.read_sql("select * from \"Распознавание_Намерений\"", dbConnection);
df['index'] = df.index
df['index'].apply(lambda x: mapping(x,map_data))
for i in list(map_data.keys()):
    if str(map_data[i]) != 'remove':
        upd = update(intent_data)
        upd = upd.values({"Класс": str(map_data[i])})
        upd = upd.where(intent_data.c.ID_Распознавание_Намерений == int(i))
        alchemyEngine.execute(upd)
        del upd
    else:
        dele = intent_data.delete().where(intent_data.c.ID_Распознавание_Намерений == int(i))
        alchemyEngine.execute(dele)
        del dele       

df = pd.read_sql("select * from \"Распознавание_Намерений\"", dbConnection);
df=df.drop(columns=['ID_Распознавание_Намерений','ID_Пользователя'])
df.columns=['Data','Class']
df['Cleared'] = df['Data'].apply(lambda col: preprocessing(col,cleared_stop_words_intent))  

qna = pd.read_sql("select * from \"ОтветыВопросы\"", dbConnection);
qna = qna.drop(columns=['id_Вопроса'])
stop_qna_words = ['Добрый','день','вечер','ночь','хотим','Здраствуйте','Привет','с','лол','и',
                  'не','эх','хотеться','бы','надо','вы','найти','можно','у','а','спосить','мне',
                  'вами','ли','чем','скажите','узнать','понять','по','больше','про','к','в','ваш'
                  'на','быть','знать', 'но','завтра','о','который',]
stop_qna_words_clear = []
for word in stop_qna_words:
    stop_qna_words_clear.append(morph.normal_forms(word)[0])
qna['Cleared'] = qna['Вопрос'].apply(lambda col: preprocessing(col,stop_qna_words_clear))

dbConnection.close();

### Словари и датасеты

In [9]:
#Подкючение к бдУ
db = context_storage_factory("XXX")#удален путь к БД

applicant_dict = {
    0: 'Если вы хотите найти работу при помощи нашего агентства вы можете либо связаться с нашими сотрудниками в рабочее время, либо оформить анкету через меня в любое время суток.\nСоглашаясь на анкетирование вы так же соглашаетесь на обработку ваших персональных данных.\nХотели ли бы вы заполнить анкету?(Да/Нет)',
    1:['Введите ваше ФИО:\n(Пример: Миронов Ярослав Викторович)','name'],
    2:['Введите ваш пол:\n(Пример: Мужчина)','gender'],
    3:['Введите дату рождения:\n (Пример: 30.09.1997)','birthdate'],
    4:['Введите вакансию:\n(Пример:Программист-стажер 1С)','profession'],
    5:['Введите ваш стаж:\n(Без опыта/х лет)','experience'],
    6:['Введите ваше основное образование:\n(Тип образования и наименование учереждения)','education'],
    7:['Введите желаемый оклад:\n(Число в рублях)','salary'],
    8:['Желаемый тип занятости(полная/частичная/стажировка):','employment_type'],
    9:['Введите ваши контактные данные:','contacts']}
employer_dict = {
    0:'Если вы работодатель и хотите при помощи нашего анетства найти работников вы можете либо связаться с нашими сотрудниками в рабочее время, либо оформить анкету через меня в любое время суток.\nСоглашаясь на анкетирование вы так же соглашаетесь на обработку ваших персональных данных.\nХотели ли бы вы заполнить анкету?(Да/Нет).',
    1:['Введите ваше полное ФИО:','ename'],
    2:['Введите название организации, которую вы представляете:','orgname'],
    3:['Введите контактные данные по котом мы сможем с вами связаться:','econtacts'],
    4:['Введите должность, на которую вы ищите работника:','eprofession'],
    5:['Введите примерную заработную плату, которую вы планируете выделить на сотрудника:','esalary'],
    6:['Введите требования к сотруднику на искомой должности','demands']}

## Модели
#### Интент

In [10]:
X = df['Cleared']
vectorizer = TfidfVectorizer()
vectorizer = vectorizer.fit(X)
X = vectorizer.transform(X)
y = df['Class']

In [11]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
intent_model = LogisticRegression(class_weight='balanced',max_iter=100,multi_class='ovr',penalty='l2',solver= 'newton-cg',tol=0.001)
intent_model.fit(X,y)

LogisticRegression(class_weight='balanced', multi_class='ovr',
                   solver='newton-cg', tol=0.001)

In [12]:
#y_pred = intent_model.predict(X_test)
#Proba_Check("Добрый день")

#### Toxicity

In [13]:
# load tokenizer and model weights
tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
tox_model = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

def toxicity_check(check):
    result = tox_model(tokenizer.encode(check, return_tensors='pt'))
    tensor = result.logits
    tensor_as_list = tensor.tolist()[0]
    print(' Neutral: ',tensor_as_list[0],' Negative:',tensor_as_list[1])

check = ''
while check != "stop":
    check = input()
    toxicity_check(check)


## Сценарий

In [14]:

script = { # все сценарии
     GLOBAL: {
        TRANSITIONS: {
            ("toxic_flow","node1",1.1): toxicity_check,
            intent_check: cnd.true()
        },
    },
    "global_flow":{ # начало стартового сценария
        "start_node":{  
            PRE_RESPONSE_PROCESSING: {"proc_name_1": ir_data_collection},
            RESPONSE: rsp.choice([
                Message(text='Чего бы вы хотели?'),
                Message(text='Как я могу вам помочь?'),
                Message(text='Что вас интересует?'),
                Message(text='Что я могу для вас сделать?'),
            ]),
        },        
        "fallback_node":{  # нод в который попадает разговор при oos сценарии
            PRE_RESPONSE_PROCESSING: {"proc_name_1": ir_data_collection},
            RESPONSE: Message(text='Что то пошло не так.'),
        },
        "rejection_node":{
             PRE_RESPONSE_PROCESSING:{"proc_name_1": remove_misc },
             RESPONSE: Message(text ='Мне очень жаль, что я не смогла вам помочь, если у вас есть какие либо вопросы или же вы все же хотите связаться через меня с нашим агентством, я всегда рада помочь!')
         },
         "success_node":{
             RESPONSE:Message(text ='Благодарим за сотрудничество! Вскоре наши операторы с вами свяжутся. Если у вас остались какие либо вопросы можете их задать!')
        },
        "already_done":{
             RESPONSE:Message(text ='В нашей базе уже хранятся ваши данные, если хотите изменить их свяжитесь с нашими сотрудниками любым удобным вам способом')
        },
    },  
    
    "applicant_flow":{
        'start':{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": ir_data_collection},
            RESPONSE: Message(text = applicant_dict[0]),
            TRANSITIONS:{
                ("global_flow","already_done"): type_check,
                lbl.forward(): cnd.regexp(r'\bда\b',re.IGNORECASE),
                ("global_flow","start_node"): cnd.regexp(r'\bнет\b',re.IGNORECASE)
            }
        },
        "name":{
            RESPONSE: Message(text=f'В любое время вы можете остановить анкетирование написав "Стоп". \n{applicant_dict[1][0]}'),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true()},
        },
          "gender":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": misc_item_catcher},
            RESPONSE: Message(text = applicant_dict[2][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true()},
        },  
        "birthdate":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": misc_item_catcher},
            RESPONSE: Message(text = applicant_dict[3][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true()},
        },  
        "profession":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": misc_item_catcher},
            RESPONSE: Message(text=applicant_dict[4][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true()},
        },
        "experience":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": misc_item_catcher},
            RESPONSE: Message(text=applicant_dict[5][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true()},
        },
        "education":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": misc_item_catcher},
            RESPONSE: Message(text=applicant_dict[6][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true()},
        },
        "salary":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": misc_item_catcher},
            RESPONSE: Message(text=applicant_dict[7][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true()},
        },
        "employment_type":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": misc_item_catcher},
            RESPONSE: Message(text=applicant_dict[8][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true(),},
        },
        "contacts":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": misc_item_catcher},
            RESPONSE: Message(text=applicant_dict[9][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true(),},
        },  
        "check":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1":misc_item_catcher},
            RESPONSE: print_applicant, #ТУТ ВЫДАЕМ ВСЕ МУЛЬТИТЕКСТОМ
            PRE_TRANSITIONS_PROCESSING:{"proc_name1": client_type},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2): cnd.regexp('нет',re.IGNORECASE),
                ("global_flow","success_node",1.2): cnd.regexp(r'\bДа\b',re.IGNORECASE),
                lbl.repeat(1.2):cnd.true()
            },
        },
        "application_change":{
            RESPONSE: Message( text='Выберите цифру, в которой ошибка'),
            TRANSITIONS:{ 
                ("applicant_flow","application_changer",1.2): num_check,
                lbl.repeat(1.2):cnd.true()
            }
        },
         "application_changer":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1":redo_key_catcher},
            RESPONSE:redo_response,
            TRANSITIONS:{("applicant_flow","check",1.2):cnd.true()}
        },
    },
    "employer_flow":{ # флоу вызываемый при распознавании желания поиска работника
            "start":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": ir_data_collection},
            RESPONSE: Message(text = employer_dict[0]),
            TRANSITIONS:{
                ("global_flow","already_done"): type_check,
                lbl.forward(): cnd.regexp(r'да',re.IGNORECASE),
                ("global_flow","start_node"): cnd.regexp(r'\bнет\b',re.IGNORECASE)
            },    
            },
            "ename":{
            RESPONSE: Message(text=f'В любое время вы можете остановить анкетирование написав "Стоп". \n{employer_dict[1][0]}'),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true(),
            }  
            },
            
            "orgname":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1":misc_item_catcher},
            RESPONSE: Message(text = employer_dict[2][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true(),
            }  
            },
            "econtacts":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1":misc_item_catcher},
            RESPONSE: Message(text = employer_dict[3][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true(),
            }  
            },
                
            "eprofession":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1":misc_item_catcher},
            RESPONSE: Message(text = employer_dict[4][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true(),
            }  
            },
                
            "esalary":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1":misc_item_catcher},
            RESPONSE: Message(text = employer_dict[5][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true(),
            }  
            },
                
            "demands":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1":misc_item_catcher},
            RESPONSE: Message(text = employer_dict[6][0]),
            PRE_TRANSITIONS_PROCESSING: {"proc_name_1": misc_key_catcher},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2):cnd.true(),
            }  
            },

            "check":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1":misc_item_catcher},
            RESPONSE: print_employer,
            PRE_TRANSITIONS_PROCESSING:{"proc_name1": client_type},
            TRANSITIONS:{
                ("global_flow","rejection_node",1.2): cnd.regexp(r'\bстоп\b', re.IGNORECASE),
                lbl.forward(1.2): cnd.regexp('нет',re.IGNORECASE),
                ("global_flow","success_node",1.2): cnd.regexp(r'\bДа\b',re.IGNORECASE),
                lbl.repeat(1.2):cnd.true()
                
            },
        },
            "application_change":{
            RESPONSE: Message( text='Выберите цифру, в которой ошибка'),
            TRANSITIONS:{ 
                ("employer_flow","application_changer",1.2): num_check,
                lbl.repeat(1.2):cnd.true()} # СДЕЛАТЬ ТУТ ЧЕК
        },   
            "application_changer":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1":redo_key_catcher_emp},
            RESPONSE:redo_response_emp,
            TRANSITIONS:{("employer_flow","check",1.2):cnd.true()}
        },   
                       
    },   
    
    
    #Флоу ответов на вопросы
    "qna_flow":{ 
        "start":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": ir_data_collection},
            RESPONSE: qna_func
        },       
    },
    
    #Флоу неподабающего поведения
    "toxic_flow":{
        "node1":{
            PRE_RESPONSE_PROCESSING: {"proc_name_1": toxic_check},
            RESPONSE: Message (text = "Пожалуйста не использйте оскорбительную речь и ненормативную лексику в диалоге со мной. Я продолжу работать с вами, но запомню, ваш поступок.")
        }
    },
}


In [15]:
actor = Actor(
    script,
    start_label=("global_flow", "start_node"),
    fallback_label=("global_flow", "fallback_node"),
)
from dff.messengers.common.interface import CallbackMessengerInterface
messenger_interface = CallbackMessengerInterface()
pipeline = Pipeline({"components": [actor]}, context_storage=db,messenger_interface = messenger_interface)

In [ ]:
bot = Bot(token='XXX')#удален токен
dp = Dispatcher(bot)
@dp.message_handler(commands=['start'])
async def process_start_command(message: types.Message):
    await message.reply("Добрый день!\nМы всегда рады новым клиентам.")
    
@dp.message_handler()
async def route(msg: types.Message):
    text = {"text": msg.text}
    await bot.send_message(msg.from_user.id, messenger_interface.on_request(text ,msg.from_user.id).last_response.text)

if __name__ == '__main__':
    pipeline.run()
    executor.start_polling(dp)